分组统计 - groupby功能

* 根据某些条件将数据拆分成组
* 对每个组独立应用函数
* 将结果合并到一个数据结构中

Dataframe在行（axis=0）或列（axis=1）上进行分组，将一个函数应用到各个分组并产生一个新值，然后函数执行结果被合并到最终的结果对象中。

df.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, **kwargs)

In [2]:
import numpy as np
import pandas as pd  
# 导入numpy、pandas模块

In [3]:
# 分组

df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
print(df)
print('---------------------')

print(df.groupby('A'),type(df.groupby('A')))
print('---------------------')
# 直接分组得到一个groupby对象，是一个中间数据，没有进行计算

a = df.groupby('A').mean()
b = df.groupby(['A','B']).mean()
c = df.groupby(['A'])['D'].mean() # 以A分组，算D的平均值
print(a,type(a),'\n',a.columns)
print(b,type(b),'\n',b.columns)
print(c,type(c))
# 通过分组后的计算，得到一个新的dataframe
# 默认axis = 0，以行来分组
# 可单个或多个（[]）列分组

     A      B         C         D
0  foo    one -0.102889  0.771421
1  bar    one  3.129709  1.505033
2  foo    two -0.766458  1.270974
3  bar  three  1.147920 -0.488897
4  foo    two -0.904889  0.642164
5  bar    two -0.360267  0.361203
6  foo    one -1.785143 -1.797042
7  foo  three  1.293075  0.464214
---------------------
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x000002472C90AC88> <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
---------------------
            C         D
A                      
bar  1.305787  0.459113
foo -0.453261  0.270346 <class 'pandas.core.frame.DataFrame'> 
 Index(['C', 'D'], dtype='object')
                  C         D
A   B                        
bar one    3.129709  1.505033
    three  1.147920 -0.488897
    two   -0.360267  0.361203
foo one   -0.944016 -0.512811
    three  1.293075  0.464214
    two   -0.835673  0.956569 <class 'pandas.core.frame.DataFrame'> 
 Index(['C', 'D'], dtype='object')
A
bar    0.459113
foo    0.270346
N

In [4]:
# 分组 - 可迭代对象

df = pd.DataFrame({'X' : ['A', 'B', 'A', 'B'], 'Y' : [1, 4, 3, 2]})
print(df)
print(df.groupby('X'), type(df.groupby('X')))
print('-----')

print(list(df.groupby('X')), '→ 可迭代对象，直接生成list\n')
print(list(df.groupby('X'))[0], '→ 以元祖形式显示\n')
for n,g in df.groupby('X'):
    print(n)
    print(g)
    print('###')
print('-----')
# n是组名，g是分组后的Dataframe

print(df.groupby(['X']).get_group('A'),'\n')
print(df.groupby(['X']).get_group('B'),'\n')
print('-----')
# .get_group()提取分组后的组

grouped = df.groupby(['X'])
print(grouped.groups)
print(grouped.groups['A'])  # 也可写：df.groupby('X').groups['A']
print('-----')
# .groups：将分组后的groups转为dict
# 可以字典索引方法来查看groups里的元素

sz = grouped.size()
print(sz,type(sz))
print('-----')
# .size()：查看分组后的长度

df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
grouped = df.groupby(['A','B']).groups
print(df)
print(grouped)
print(grouped[('foo', 'three')])
# 按照两个列进行分组

   X  Y
0  A  1
1  B  4
2  A  3
3  B  2
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x000002472C92EEB8> <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
-----
[('A',    X  Y
0  A  1
2  A  3), ('B',    X  Y
1  B  4
3  B  2)] → 可迭代对象，直接生成list

('A',    X  Y
0  A  1
2  A  3) → 以元祖形式显示

A
   X  Y
0  A  1
2  A  3
###
B
   X  Y
1  B  4
3  B  2
###
-----
   X  Y
0  A  1
2  A  3 

   X  Y
1  B  4
3  B  2 

-----
{'A': Int64Index([0, 2], dtype='int64'), 'B': Int64Index([1, 3], dtype='int64')}
Int64Index([0, 2], dtype='int64')
-----
X
A    2
B    2
dtype: int64 <class 'pandas.core.series.Series'>
-----
     A      B         C         D
0  foo    one  1.242631  0.396043
1  bar    one  0.109723 -0.213725
2  foo    two  0.636748 -0.905796
3  bar  three -2.031638 -0.849169
4  foo    two -0.398462 -1.030888
5  bar    two -0.144023 -1.326317
6  foo    one  0.136222 -1.697496
7  foo  three  2.823601  0.543055
{('bar', 'one'): Int64Index([1], dtype='int64'), ('bar', 'three'): Int64Inde

In [5]:
# 分组计算函数方法

s = pd.Series([1, 2, 3, 10, 20, 30], index = [1, 2, 3, 1, 2, 3])
grouped = s.groupby(level=0)  # 唯一索引用.groupby(level=0)，将同一个index的分为一组
print(grouped)
print(grouped.first(),'→ first：非NaN的第一个值\n')
print(grouped.last(),'→ last：非NaN的最后一个值\n')
print(grouped.sum(),'→ sum：非NaN的和\n')
print(grouped.mean(),'→ mean：非NaN的平均值\n')
print(grouped.median(),'→ median：非NaN的算术中位数\n')
print(grouped.count(),'→ count：非NaN的值\n')
print(grouped.min(),'→ min、max：非NaN的最小值、最大值\n')
print(grouped.std(),'→ std，var：非NaN的标准差和方差\n')
print(grouped.prod(),'→ prod：非NaN的积\n')

1    1
2    2
3    3
dtype: int64 → first：非NaN的第一个值

1    10
2    20
3    30
dtype: int64 → last：非NaN的最后一个值

1    11
2    22
3    33
dtype: int64 → sum：非NaN的和

1     5.5
2    11.0
3    16.5
dtype: float64 → mean：非NaN的平均值

1     5.5
2    11.0
3    16.5
dtype: float64 → median：非NaN的算术中位数

1    2
2    2
3    2
dtype: int64 → count：非NaN的值

1    1
2    2
3    3
dtype: int64 → min、max：非NaN的最小值、最大值

1     6.363961
2    12.727922
3    19.091883
dtype: float64 → std，var：非NaN的标准差和方差

1    10
2    40
3    90
dtype: int64 → prod：非NaN的积



In [6]:
# 多函数计算：agg()

df = pd.DataFrame({'a':[1,1,2,2],
                  'b':np.random.rand(4),
                  'c':np.random.rand(4),
                  'd':np.random.rand(4),})
print(df)
print(df.groupby('a').agg(['mean',np.sum]))
print(df.groupby('a')['b'].agg({'result1':np.mean,
                               'result2':np.sum}))
# 函数写法可以用str，或者np.方法
# 可以通过list，dict传入，当用dict时，key名为columns → 更新pandas后会出现警告
# 尽量用list传入

   a         b         c         d
0  1  0.474520  0.068500  0.146103
1  1  0.754903  0.109037  0.973589
2  2  0.400874  0.138578  0.308826
3  2  0.693645  0.674025  0.060644
          b                   c                   d          
       mean       sum      mean       sum      mean       sum
a                                                            
1  0.614712  1.229423  0.088769  0.177537  0.559846  1.119693
2  0.547259  1.094518  0.406301  0.812603  0.184735  0.369470
    result1   result2
a                    
1  0.614712  1.229423
2  0.547259  1.094518


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


######## 课后小练习,请查看 “pandas课程作业.docx”  ########

作业1：按要求创建Dataframe df，并通过分组得到以下结果
* 以A分组，求出C,D的分组平均值
* 以A,B分组，求出D,E的分组求和
* 以A分组，得到所有分组，以字典显示
* 按照数值类型分组，求和
* 将C,D作为一组分出来，并计算求和
* 以B分组，求出每组的均值，求和，最大值，最小值

In [8]:
df = pd.DataFrame({'A':['one','two','three','one','two','three','one','two'],
                  'B':list('hhhhffff'),
                  'C':list(range(10,25,2)),
                  'D':np.random.randn(8),
                  'E':np.random.randn(8)})
df

,A,B,C,D,E
0,one,h,10,1.123990,0.462437
1,two,h,12,-0.214674,-0.317127
2,three,h,14,-0.627604,-1.131893
3,one,h,16,0.238987,0.436619
4,two,f,18,0.270927,-0.260119
5,three,f,20,0.472409,-0.199017
6,one,f,22,0.001404,1.068322
7,two,f,24,-2.033420,0.268944


In [9]:
df.groupby(by='A')[['D','E']].mean()

,D,E
A,,
one,0.454794,0.655793
three,-0.077597,-0.665455
two,-0.659056,-0.102767


In [10]:
df.groupby(by=['A','B'])[['D','E']].sum()

D         E
A     B                    
one   f  0.001404  1.068322
      h  1.362977  0.899056
three f  0.472409 -0.199017
      h -0.627604 -1.131893
two   f -1.762493  0.008825
      h -0.214674 -0.317127

In [13]:
g = df.groupby(by='A')
for n,dfi in g:
    print(n)
    print(dfi)

one
     A  B   C         D         E
0  one  h  10  1.123990  0.462437
3  one  h  16  0.238987  0.436619
6  one  f  22  0.001404  1.068322
three
       A  B   C         D         E
2  three  h  14 -0.627604 -1.131893
5  three  f  20  0.472409 -0.199017
two
     A  B   C         D         E
1  two  h  12 -0.214674 -0.317127
4  two  f  18  0.270927 -0.260119
7  two  f  24 -2.033420  0.268944


In [14]:
df.groupby(df.dtypes,axis=1).sum()

,int64,float64,object
0,10,1.586427,oneh
1,12,-0.531801,twoh
2,14,-1.759497,threeh
3,16,0.675606,oneh
4,18,0.010808,twof
5,20,0.273392,threef
6,22,1.069726,onef
7,24,-1.764476,twof


In [15]:
df2 = df[['C','D']]
df2['sum'] = df2.sum(axis=1)
df2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,C,D,sum
0,10,1.123990,11.123990
1,12,-0.214674,11.785326
2,14,-0.627604,13.372396
3,16,0.238987,16.238987
4,18,0.270927,18.270927
5,20,0.472409,20.472409
6,22,0.001404,22.001404
7,24,-2.033420,21.966580


In [16]:
df.groupby('B').agg(['mean','sum','max',np.min])

C                      D                                       E  \
  mean sum max amin      mean       sum       max      amin      mean   
B                                                                       
f   21  84  24   18 -0.322170 -1.288680  0.472409 -2.033420  0.219532   
h   13  52  16   10  0.130175  0.520699  1.123990 -0.627604 -0.137491   

                                 
        sum       max      amin  
B                                
f  0.878130  1.068322 -0.260119  
h -0.549964  0.462437 -1.131893